## Azure OpenAI GPT-V Target Demo
This notebook demonstrates how to use the Azure OpenAI GPT-V target to accept multimodal input (text+image) and generate text output.

In [1]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT license.


from pyrit.models import PromptRequestPiece, PromptRequestResponse
from pyrit.prompt_target import AzureOpenAIGPTVChatTarget
from pyrit.common import default_values
import pathlib
from pyrit.common.path import HOME_PATH
import uuid

default_values.load_default_env()
test_conversation_id = str(uuid.uuid4())

# use the image from our docs
image_path = pathlib.Path(HOME_PATH) / "assets" / "pyrit_architecture.png"

request_pieces = [
    PromptRequestPiece(
        role="user",
        conversation_id=test_conversation_id,
        original_value="Describe this picture:",
        original_value_data_type="text",
        converted_value_data_type="text",
    ),
    PromptRequestPiece(
        role="user",
        conversation_id=test_conversation_id,
        original_value=str(image_path),
        original_value_data_type="image_path",
        converted_value_data_type="image_path",
    ),
]

In [2]:
prompt_request_response = PromptRequestResponse(request_pieces=request_pieces)

In [3]:
with AzureOpenAIGPTVChatTarget() as azure_openai_chat_target:
    resp = await azure_openai_chat_target.send_prompt_async(prompt_request=prompt_request_response)  # type: ignore
    print(resp)

None: assistant: The image displays a chart titled "PyRIT Components," which appears to outline the structure of a system or framework named PyRIT. The chart is divided into two columns: Interface and Implementation.

Under the Interface column, there are five categories:
1. Target
2. Datasets
3. Scoring Engine
4. Attack Strategy
5. Memory

Adjacent to each Interface category, there's an Implementation description:
- For Target, there are two types: Local (local model e.g., ONNX) and Remote (API or web app).
- Datasets can be either Static (prompts) or Dynamic (Prompt templates).
- The Scoring Engine's implementation is described as PyRIT Itself: Self Evaluation and API: Existing content classifiers.
- Attack Strategy can be Single Turn (Using static prompts) or Multi Turn (Multiple conversations using prompt templates).
- Memory is detailed with Storage (JSON, Database) and Utils (Conversation, retrieval and storage, memory sharing, data analysis).

Overall, the image seems to summari

## Multimodal Demo using AzureOpenAIGPTVChatTarget and PromptSendingOrchestrator
This demo showcases the capabilities of AzureOpenAIGPTVChatTarget for generating text based on multimodal inputs, including both text and images using PromptSendingOrchestrator.

In [ ]:
from pyrit.common import default_values
import pathlib
from pyrit.common.path import HOME_PATH

from pyrit.prompt_target import AzureOpenAIGPTVChatTarget
from pyrit.prompt_normalizer.normalizer_request import NormalizerRequestPiece
from pyrit.prompt_normalizer.normalizer_request import NormalizerRequest
from pyrit.orchestrator import PromptSendingOrchestrator

default_values.load_default_env()

azure_openai_gptv_chat_target = AzureOpenAIGPTVChatTarget()

image_path = pathlib.Path(HOME_PATH) / "assets" / "pyrit_architecture.png"
data = [
    [
        {"prompt_text": "Describe this picture:", "prompt_data_type": "text"},
        {"prompt_text": str(image_path), "prompt_data_type": "image_path"},
    ],
    [{"prompt_text": "Tell me about something?", "prompt_data_type": "text"}],
    [{"prompt_text": str(image_path), "prompt_data_type": "image_path"}],
]


normalizer_requests = []

for piece_data in data:
    request_pieces = []

    for item in piece_data:
        prompt_text = item.get("prompt_text", "")  # type: ignore
        prompt_data_type = item.get("prompt_data_type", "")
        converters = []  # type: ignore
        request_piece = NormalizerRequestPiece(
            prompt_value=prompt_text, prompt_data_type=prompt_data_type, request_converters=converters  # type: ignore
        )
        request_pieces.append(request_piece)

    normalizer_request = NormalizerRequest(request_pieces)
    normalizer_requests.append(normalizer_request)


    
with PromptSendingOrchestrator(prompt_target=azure_openai_gptv_chat_target) as orchestrator:

    await orchestrator.send_normalizer_requests_async(prompt_request_list=normalizer_requests)  # type: ignore

    memory = orchestrator.get_memory()

    for entry in memory:
        print(entry)